# 2. Preprocesamiento de Datos y Feature Engineering

**Objetivo:** Preparar los datos para el entrenamiento de modelos de Machine Learning. Basado en los hallazgos del EDA, realizaremos la ingeniería de características, división de datos y el escalado de variables numéricas.

## 2.1. Carga de Datos y Librerías

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import os

# --- INICIO: Celda de configuración para guardado --- 
# 1. Crear carpetas para los reportes
FIGS_PATH = "../reports/figures/preprocessing"
METRICS_PATH = "../reports/metrics"
os.makedirs(FIGS_PATH, exist_ok=True)
os.makedirs(METRICS_PATH, exist_ok=True)

# 2. Inicializar contadores
nuevos_graficos = 0
graficos_omitidos = 0
nuevas_tablas = 0
tablas_omitidas = 0

print(f"Directorios listos en: {FIGS_PATH} y {METRICS_PATH}")
print("Contadores inicializados.")

# Cargar datos
DATA_PATH = "../data/processed/dataset_clinico_huancayo_20k_processed.csv"
df = pd.read_csv(DATA_PATH)

# Mapeo de diagnóstico para referencia
diagnostico_map = {0: 'DM2', 1: 'EDA', 2: 'HTA', 3: 'IRA'}
df['diagnostico_str'] = df['diagnostico'].map(diagnostico_map)

display(df.head())

Directorios listos en: ../reports/figures/preprocessing y ../reports/metrics
Contadores inicializados.


,id,edad,sexo,area,distrito,ocupacion,imc,pas,pad,fc,...,sintoma_palpitaciones,sintoma_vomitos,sintoma_malestar_general,sintoma_mareo,sintoma_dolor_garganta,sintoma_epistaxis,sintoma_congestion_nasal,sintoma_fatiga,sintoma_dolor_pecho,diagnostico_str
0,1,12,0,1,2,8,23.8,127,75,94,...,0,0,1,0,0,0,0,0,1,IRA
1,2,19,0,1,7,4,30.5,110,73,98,...,0,0,0,0,0,0,0,0,0,EDA
2,3,51,0,1,3,5,27.9,126,76,84,...,0,0,0,0,0,0,0,0,0,DM2
3,4,44,0,1,4,7,34.9,157,110,92,...,0,0,0,0,0,1,0,1,0,HTA
4,5,14,0,1,9,8,22.2,119,81,103,...,0,0,1,0,1,0,1,0,1,IRA


## 2.2. Ingeniería de Características (Feature Engineering)

Crearemos variables adicionales que pueden capturar mejor las relaciones clínicas.
- **Presión de Pulso (PP):** `pas - pad`. Es un indicador de riesgo cardiovascular.
- **Categorías de IMC:** Clasificar el IMC en bajo peso, normal, sobrepeso y obesidad.

In [12]:
# Presión de Pulso
df['presion_pulso'] = df['pas'] - df['pad']

# Categorías de IMC
imc_bins = [0, 18.5, 24.9, 29.9, np.inf]
imc_labels = [0, 1, 2, 3]  # 0: Bajo peso, 1: Normal, 2: Sobrepeso, 3: Obesidad
df['imc_categoria'] = pd.cut(df['imc'], bins=imc_bins, labels=imc_labels, right=False)

print("Nuevas características creadas:")
new_features_df = df[['pas', 'pad', 'presion_pulso', 'imc', 'imc_categoria']].head()
display(new_features_df)

# Guardar sample de nuevas características
table_path = os.path.join(METRICS_PATH, "nuevas_caracteristicas_head.csv")
if not os.path.exists(table_path):
    new_features_df.to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

Nuevas características creadas:


,pas,pad,presion_pulso,imc,imc_categoria
0,127,75,52,23.8,1
1,110,73,37,30.5,3
2,126,76,50,27.9,2
3,157,110,47,34.9,3
4,119,81,38,22.2,1


⏭️ Archivo nuevas_caracteristicas_head.csv ya existe, omitiendo...


## 2.3. División de Datos (Train-Test Split)

Dividimos el dataset en un conjunto de entrenamiento (80%) y uno de prueba (20%). Es **crítico** usar la estratificación (`stratify`) en la variable objetivo (`diagnostico`) para asegurar que la proporción de cada enfermedad sea la misma en ambos conjuntos. Esto es especialmente importante dado el desbalance de clases observado en el EDA.

In [13]:
# Separar características (X) y variable objetivo (y)
X = df.drop(['id', 'diagnostico', 'diagnostico_str', 'sexo_str', 'area_str'], axis=1, errors='ignore')
y = df['diagnostico']

# División estratificada
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, # Para reproducibilidad
    stratify=y
)

print(f'Tamaño de X_train: {X_train.shape}')
print(f'Tamaño de X_test: {X_test.shape}')

# Guardar tamaños de split
split_sizes_df = pd.DataFrame({'X_train_shape':[X_train.shape], 'X_test_shape':[X_test.shape]})
table_path = os.path.join(METRICS_PATH, "sizes_split.csv")
if not os.path.exists(table_path):
    split_sizes_df.to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

# Distribución en y_train
print("Distribución de diagnósticos en y_train:")
dist_y_train = y_train.value_counts(normalize=True) * 100
display(dist_y_train)
table_path = os.path.join(METRICS_PATH, "distribucion_y_train.csv")
if not os.path.exists(table_path):
    dist_y_train.reset_index().rename(columns={'index':'diagnostico','diagnostico': 'percentage'}).to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

# Distribución en y_test
print("Distribución de diagnósticos en y_test:")
dist_y_test = y_test.value_counts(normalize=True) * 100
display(dist_y_test)
table_path = os.path.join(METRICS_PATH, "distribucion_y_test.csv")
if not os.path.exists(table_path):
    dist_y_test.reset_index().rename(columns={'index':'diagnostico','diagnostico': 'percentage'}).to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

Tamaño de X_train: (16000, 54)
Tamaño de X_test: (4000, 54)
⏭️ Archivo sizes_split.csv ya existe, omitiendo...
Distribución de diagnósticos en y_train:


3    30.0
2    28.0
0    24.0
1    18.0
Name: diagnostico, dtype: float64

⏭️ Archivo distribucion_y_train.csv ya existe, omitiendo...
Distribución de diagnósticos en y_test:


3    30.0
2    28.0
0    24.0
1    18.0
Name: diagnostico, dtype: float64

⏭️ Archivo distribucion_y_test.csv ya existe, omitiendo...


## 2.4. Escalado de Características Numéricas

Los modelos como SVM y Redes Neuronales son sensibles a la escala de las características. Usaremos `StandardScaler` para estandarizar las variables numéricas (media 0, desviación estándar 1).

**Importante:** El escalador se ajusta (`fit`) **únicamente** con los datos de entrenamiento para evitar fuga de información del conjunto de prueba. Luego, se aplica la transformación a ambos conjuntos (entrenamiento y prueba).

In [14]:
# Identificar columnas numéricas (excluyendo las binarias/codificadas que no necesitan escalado)
numerical_cols = ['edad', 'imc', 'pas', 'pad', 'fc', 'fr', 'temp', 'spo2', 'glucosa', 'hba1c', 'creatinina', 'colesterol', 'leucocitos', 'tiempo_enfermedad', 'presion_pulso']

# Crear y ajustar el escalador
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])

# Aplicar la misma transformación al conjunto de prueba
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

print("Características numéricas estandarizadas. Mostrando la media y desviación estándar de X_train después del escalado:")
scaled_stats = X_train[numerical_cols].describe().T[['mean', 'std']]
display(scaled_stats)

# Guardar estadísticas de escalado
table_path = os.path.join(METRICS_PATH, "estadisticas_escalado_X_train.csv")
if not os.path.exists(table_path):
    scaled_stats.reset_index().to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

Características numéricas estandarizadas. Mostrando la media y desviación estándar de X_train después del escalado:


,mean,std
edad,7.305268e-17,1.000031
imc,4.041212e-16,1.000031
pas,1.707523e-16,1.000031
pad,2.238210e-16,1.000031
fc,8.260059e-17,1.000031
fr,1.803002e-16,1.000031
temp,-2.859490e-15,1.000031
spo2,-2.415845e-16,1.000031
glucosa,3.064216e-17,1.000031
hba1c,1.136868e-16,1.000031


⏭️ Archivo estadisticas_escalado_X_train.csv ya existe, omitiendo...


## 2.5. Guardar los Datos Procesados y el Escalador

Finalmente, guardamos los dataframes procesados y el objeto `scaler`. Esto nos permitirá cargarlos directamente en el notebook de modelado sin tener que repetir los pasos de preprocesamiento.

In [15]:
# Crear directorio para los datos procesados si no existe
PROCESSED_DATA_DIR = "../data/processed/"
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

# Guardar dataframes
X_train.to_csv(os.path.join(PROCESSED_DATA_DIR, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(PROCESSED_DATA_DIR, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(PROCESSED_DATA_DIR, 'y_test.csv'), index=False)

# Guardar el escalador
MODELS_DIR = "../models/"
os.makedirs(MODELS_DIR, exist_ok=True)
joblib.dump(scaler, os.path.join(MODELS_DIR, 'scaler.pkl'))

print(f"Datos procesados guardados en: {PROCESSED_DATA_DIR}")
print(f"Escalador guardado en: {os.path.join(MODELS_DIR, 'scaler.pkl')}")

# Guardar también muestras de X_train/X_test en reports/metrics
# X_train head
table_path = os.path.join(METRICS_PATH, "X_train_head.csv")
if not os.path.exists(table_path):
    X_train.head().to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

# X_test head
table_path = os.path.join(METRICS_PATH, "X_test_head.csv")
if not os.path.exists(table_path):
    X_test.head().to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

# y_train head
table_path = os.path.join(METRICS_PATH, "y_train_head.csv")
if not os.path.exists(table_path):
    y_train.reset_index(drop=True).to_frame(name='diagnostico').head().to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

# y_test head
table_path = os.path.join(METRICS_PATH, "y_test_head.csv")
if not os.path.exists(table_path):
    y_test.reset_index(drop=True).to_frame(name='diagnostico').head().to_csv(table_path, index=False)
    nuevas_tablas += 1
else:
    print(f"⏭️ Archivo {os.path.basename(table_path)} ya existe, omitiendo...")
    tablas_omitidas += 1

Datos procesados guardados en: ../data/processed/
Escalador guardado en: ../models/scaler.pkl
⏭️ Archivo X_train_head.csv ya existe, omitiendo...
⏭️ Archivo X_test_head.csv ya existe, omitiendo...
⏭️ Archivo y_train_head.csv ya existe, omitiendo...
⏭️ Archivo y_test_head.csv ya existe, omitiendo...


In [16]:
# --- INICIO: Celda de resumen --- 
print("✅ Proceso completo:")
print(f"- {nuevos_graficos} gráficos nuevos guardados en {FIGS_PATH}")
print(f"- {graficos_omitidos} gráficos omitidos (ya existían).")
print(f"- {nuevas_tablas} tablas nuevas guardadas en {METRICS_PATH}")
print(f"- {tablas_omitidas} tablas omitidas (ya existían).")

✅ Proceso completo:
- 0 gráficos nuevos guardados en ../reports/figures/preprocessing
- 0 gráficos omitidos (ya existían).
- 0 tablas nuevas guardadas en ../reports/metrics
- 9 tablas omitidas (ya existían).
